In [19]:
import tifffile
import numpy as np
from pathlib import Path
from tqdm import tqdm
import os
from skimage import measure
from skimage.measure import regionprops

pip install fill-voids

In [2]:
def read_sequence(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.

    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted(
        [
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if (f.endswith(".tiff") or f.endswith(".tif"))
        ]
    )

    tiff_sequence = tifffile.TiffSequence(tiff_files)

    # Get the total number of TIFF files
    total_files = len(tiff_files)

    # Read each TIFF file and update progress
    volume = []
    with tqdm(total=total_files, desc="Progress") as pbar:
        for i, file_path in enumerate(tiff_files):
            slice_data = tifffile.imread(file_path)
            volume.append(slice_data)

            # Update progress
            pbar.update(1)

    return np.array(volume)

In [3]:
def write_sequence(folder_path, name, volume):
    """
    Save a 3D volume as a sequence of TIFF files in a folder.

    Args:
    folder_path (str): Path to the folder where TIFF files will be saved.
    name (str): Name of the TIFF files.
    volume (numpy.ndarray): A 3D array where each slice corresponds to an image.
    """

    folder_path = folder_path / name

    # Create the folder if it doesn't exist
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    # Save each slice as a TIFF file with progress bar
    with tqdm(total=volume.shape[0], desc="Saving") as pbar:
        for i in range(volume.shape[0]):
            tifffile.imwrite(f"{folder_path}/{name}_{i:04d}.tif", volume[i])
            pbar.update(1)

    print("Saving complete.")

In [6]:
xct_folder = Path(
    r"Q:\11_AIRBUS\03_LabXCT\Panel Pegaso\Probetas\1\26\output\resampled_xct"
)

xct = read_sequence(xct_folder)

# xct = tifffile.imread(xct_folder / 'frontal_cropped.tif')

Progress:   0%|          | 0/350 [00:00<?, ?it/s]

Progress: 100%|██████████| 350/350 [01:03<00:00,  5.47it/s]


In [13]:
from skimage import filters

print("masking")

# Create a masked array, excluding zeros
masked_data = np.ma.masked_equal(xct, 0)

unmasked_data = masked_data.compressed()

print("computing otsu")

# Apply Otsu thresholding on the non-zero values
threshold_value = filters.threshold_otsu(unmasked_data)

print("thresholding with value: ", threshold_value)

binary = xct > threshold_value

masking
computing otsu
thresholding with value:  108


In [20]:
max_proj = np.max(binary, axis=0)

labels = measure.label(max_proj)

props = regionprops(labels)

minr, minc, maxr, maxc = props[0].bbox

# crop the volume

binary_cropped = binary[:, minr:maxr, minc:maxc]

In [22]:
import fill_voids

sample_mask_cropped = fill_voids.fill(binary_cropped, in_place=False)

In [25]:
sample_mask = np.zeros_like(binary)
sample_mask[:, minr:maxr, minc:maxc] = sample_mask_cropped

In [26]:
# invert binary
binary_inverted = np.invert(binary)
onlypores = np.logical_and(binary_inverted, sample_mask)

In [27]:
output_folder = xct_folder.parent / "segmentation"

# save the segmented volume
write_sequence(output_folder, "onlypores", onlypores.astype(np.uint8) * 255)
# save the sample mask
write_sequence(output_folder, "sample_mask", sample_mask.astype(np.uint8) * 255)
# save the segmented volume
write_sequence(output_folder, "binary", binary.astype(np.uint8) * 255)

Saving: 100%|██████████| 350/350 [00:56<00:00,  6.19it/s]


Saving complete.


Saving: 100%|██████████| 350/350 [00:57<00:00,  6.12it/s]


Saving complete.


Saving: 100%|██████████| 350/350 [01:14<00:00,  4.71it/s]

Saving complete.
